# Lab 04 : Test set evaluation -- exercise

In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'test_set_exercise.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    #path_to_file = '/content/gdrive/My Drive/AI6103_2020_codes/codes/labs_lecture05/lab04_test_set'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd
    

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import utils

### Download the CIFAR dataset -- check the size carefully!

In [3]:
from utils import check_cifar_dataset_exists
data_path=check_cifar_dataset_exists()

train_data=torch.load(data_path+'cifar/train_data.pt')
train_label=torch.load(data_path+'cifar/train_label.pt')
test_data=torch.load(data_path+'cifar/test_data.pt')
test_label=torch.load(data_path+'cifar/test_label.pt')

print(train_data.size())

torch.Size([50000, 3, 32, 32])


### Make a ONE layer net class. 

In [4]:
class one_layer_net(nn.Module):

    def __init__(self, input_size, output_size):
        super(one_layer_net , self).__init__()
        self.linear_layer = nn.Linear( input_size, output_size , bias=True)
        
    def forward(self, x):
        scores = self.linear_layer(x)
        return scores

### Build the net. 

In [5]:
net=one_layer_net(3072,10)
print(net)

one_layer_net(
  (linear_layer): Linear(in_features=3072, out_features=10, bias=True)
)


### Choose the criterion and optimizer. Also choose:


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD( net.parameters() , lr=0.01 )
bs=20

### Do 50 passes through the training set.

In [7]:
for epoch in range(50):
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(50000)
 
    for count in range(0,50000,bs):
    
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch       
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices]
        minibatch_label= train_label[indices]

        # reshape the minibatch
        inputs = minibatch_data.view(bs,3072)

        # tell Pytorch to start tracking all operations that will be done on "inputs"
        inputs.requires_grad_()

        # forward the minibatch through the net 
        scores=net( inputs ) 

        # Compute the average of the losses of the data points in the minibatch
        loss =  criterion( scores , minibatch_label) 
        
        # backward pass to compute dL/dU, dL/dV and dL/dW   
        loss.backward()

        # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
        optimizer.step()
        
        # START COMPUTING STATS
        
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item()
        
        # compute the error made on this batch and add it to the running error       
        error = utils.get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
    
    
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    
    print('epoch=',epoch, '\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
        
    

epoch= 0 	 loss= 1.9574754604816438 	 error= 69.52999995231627 percent
epoch= 1 	 loss= 1.8627463012695313 	 error= 65.35599997520447 percent
epoch= 2 	 loss= 1.8314529631137848 	 error= 63.9479999423027 percent
epoch= 3 	 loss= 1.818766708278656 	 error= 63.15399993181229 percent
epoch= 4 	 loss= 1.8062574334621428 	 error= 62.86000001907348 percent
epoch= 5 	 loss= 1.7972621258258819 	 error= 62.60199995994567 percent
epoch= 6 	 loss= 1.7907009477615357 	 error= 62.07999996185303 percent
epoch= 7 	 loss= 1.783532154607773 	 error= 61.89200000524521 percent
epoch= 8 	 loss= 1.7800080544471741 	 error= 61.697999944686885 percent
epoch= 9 	 loss= 1.7731269805908203 	 error= 61.37400001287461 percent
epoch= 10 	 loss= 1.7685934644222259 	 error= 61.139999992847436 percent
epoch= 11 	 loss= 1.7627871175289154 	 error= 60.7560000705719 percent
epoch= 12 	 loss= 1.763217049241066 	 error= 60.991999936103824 percent
epoch= 13 	 loss= 1.7562050785779952 	 error= 60.69599999427795 percent
epoc

# You only have to complete this cell

### So we are doing around 59% error on the training set. BUT how much are we doing on test set?

In [10]:
running_error= 0# COMPLETE HERE
num_batches= 0# COMPLETE HERE

for count in range(0, len(test_data), bs): # COMPLETE HERE

    # extract the minibatch
    minibatch_data = test_data[count:count+bs] # COMPLETE HERE
    minibatch_label= test_label[count:count+bs]  # COMPLETE HERE

    # reshape the minibatch
    inputs = minibatch_data.view(bs, 3072) # COMPLETE HERE

    # feed it to the network
    scores= net(inputs) # COMPLETE HERE

    # compute the error made on this batch
    error = utils.get_error(scores, minibatch_label)# COMPLETE HERE
    
    # add it to the running error
    running_error += error.detach().item() # COMPLETE HERE

    num_batches += 1# COMPLETE HERE


# compute error rate on the full test set
total_error = running_error / num_batches# COMPLETE HERE

print( 'error rate on test set =', total_error*100 ,'percent')

error rate on test set = 61.67999997138976 percent


### Choose image at random from the test set and see how good/bad are the predictions

In [11]:
# choose a picture at random
idx=randint(0, 10000-1)
im=test_data[idx]

# diplay the picture
utils.show(im)

# feed it to the net and display the confidence scores
scores =  net( im.view(1,3072)) 
probs= F.softmax(scores, dim=1)
utils.show_prob_cifar(probs)